In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)
conll["test"][2]

Found cached dataset conll2003 (/Users/kirillsergeev/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 484.26it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [3]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [4]:
import numpy as np
import torch
from tqdm import tqdm

class ModelWrapper():
    def __init__(self, model, tokenizer):
        self.model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None)

    def _unite_entities(self, entities):
        if len(entities) <= 1:
            return entities

        united_result = []
        cur_entity = {key: entities[0][key] for key in ['entity', 'word', 'start', 'end']}
        for entity in entities[1:]:
            if entity['word'].startswith('##'):
                cur_entity['word'] += entity['word'].lstrip('#')
                cur_entity['end'] = entity['end']
            else:
                united_result.append(cur_entity)
                cur_entity = {key: entity[key] for key in ['entity', 'word', 'start', 'end']}
        united_result.append(cur_entity)
        return united_result

    def _convert_entities_to_bio(self, tokens, entities):
        bio_tags = []
        cur_entity_idx = 0
        for token in tokens:
            if (cur_entity_idx < len(entities))\
                    and (token == entities[cur_entity_idx]['word']):
                bio_tags.append(entities[cur_entity_idx]['entity'])
                cur_entity_idx += 1
            else:
                bio_tags.append('O')
        return bio_tags

    def _postprocessing(self, tokens, model_output):
        entities = self._unite_entities(model_output)
        bio_tags = self._convert_entities_to_bio(tokens, entities)
        return bio_tags

    def predict(self, X):
        with torch.no_grad():
            ner_entitites = self.model(X)
            tags = []
            for text, entities in tqdm(zip(X, ner_entitites)):
                tags.append(self._postprocessing(text.split(), entities))
            return tags

wrapped_model = ModelWrapper(model, tokenizer)

In [5]:
def preprocessing_dataset(dataset):
    result = {'X': [], 'y_true': []}
    for sample in dataset:
        result['X'].append(' '.join(sample['tokens']))
        result['y_true'].append([CONLL_NER_TAGS[tag] for tag in sample['ner_tags']])
    return result

In [6]:
# from itertools import islice
# train_data = preprocessing_dataset(conll['train'])
# train_data['y_pred'] = wrapped_model.predict(train_data['X'])

# oos_data = preprocessing_dataset(conll['test'])
# oos_data['y_pred'] = wrapped_model.predict(oos_data['X'])

In [9]:
# import pickle

# with open('./train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
# with open('./oos_data.pkl', 'wb') as f:
#     pickle.dump(oos_data, f)

In [9]:
import pickle

with open('./train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('./oos_data.pkl', 'rb') as f:
    oos_data = pickle.load(f)

In [16]:
print(all([len(i) == len(j) for i, j in zip(train_data['y_true'], train_data['y_pred'])]))
print(all([len(i) == len(j) for i, j in zip(oos_data['y_true'], oos_data['y_pred'])]))

True
True


In [18]:
from sbe_vallib.validation.sampler.ner_sampler import NerSampler

sampler = NerSampler(train=train_data, oos=oos_data)
sampler.set_seed(42, bootstrap=True)

In [31]:
sorted(list(set([tag.split('-')[-1] for tag in CONLL_NER_TAGS]) - set(['O'])))

['LOC', 'MISC', 'ORG', 'PER']

In [50]:
from sbe_vallib.validation.scorer.ner_scorer import NerScorer
from sbe_vallib.validation.utils.metrics import NER_IOB_METRICS


scorer = NerScorer(metrics=NER_IOB_METRICS, )
scores = scorer.score(oos_data['y_true'], oos_data['y_pred'])
scores

{'precision_score': {'LOC': 0.9363057324840764,
  'MISC': 0.804920913884007,
  'ORG': 0.8944262295081967,
  'PER': 0.9319965126416739,
  'micro avg': 0.9043806982432733,
  'macro avg': 0.8919123471294885,
  'weighted avg': 0.9064257739880867},
 'f1_score': {'LOC': 0.8043775649794801,
  'MISC': 0.7206923682140047,
  'ORG': 0.8562460765850596,
  'PER': 0.7735166425470332,
  'micro avg': 0.8017742730409069,
  'macro avg': 0.7887081630813945,
  'weighted avg': 0.800394646791491},
 'recall_score': {'LOC': 0.7050359712230215,
  'MISC': 0.6524216524216524,
  'ORG': 0.8211920529801324,
  'PER': 0.6611008039579468,
  'micro avg': 0.7200779036827195,
  'macro avg': 0.7099376201456882,
  'weighted avg': 0.7200779036827195},
 'support': {'LOC': 1668,
  'MISC': 702,
  'ORG': 1661,
  'PER': 1617,
  'micro avg': 5648,
  'macro avg': 5648,
  'weighted avg': 5648}}

In [51]:
scorer.ner_metrics(oos_data['y_true'], oos_data['y_pred'])

defaultdict(dict,
            {'f1_ent_type': {'LOC': 0.8062648961525366,
              'MISC': 0.7612179487179488,
              'ORG': 0.8829452485840151,
              'PER': 0.8030194104960461,
              'macro': 0.8238541153277475},
             'f1_partial': {'LOC': 0.8478038815117467,
              'MISC': 0.7908653846153845,
              'ORG': 0.9118942731277533,
              'PER': 0.8076923076923076,
              'macro': 0.8498767865943814},
             'f1_strict': {'LOC': 0.800817160367722,
              'MISC': 0.733974358974359,
              'ORG': 0.8584015103838892,
              'PER': 0.7685118619698059,
              'macro': 0.8017742730409069},
             'f1_exact': {'LOC': 0.8409942117807286,
              'MISC': 0.767628205128205,
              'ORG': 0.8923851478917558,
              'PER': 0.7886412652767792,
              'macro': 0.8337111877772303}})

In [29]:
from sbe_vallib.validation.nlp.general_tests.model_quality.test_key_metric import test_key_metric

In [30]:
test_key_metric(wrapped_model, scorer, sampler)['result_dataframes'][0]

,tag,support,f1-score,semaphore
0,PER,1617,0.773517,green
1,MISC,702,0.720692,green
2,LOC,1668,0.804378,green
3,ORG,1661,0.856246,green


In [35]:
from seqeval.metrics import classification_report

res = classification_report(oos_data['y_true'], oos_data['y_pred'], output_dict=True)

def _ner_recall(y_true, y_pred):
    

In [37]:
from nervaluate import Evaluator

evaluator = Evaluator(oos_data['y_true'], oos_data['y_pred'], tags=['PER', 'MISC', 'LOC', 'ORG'], loader='list')
ner_metrics, ner_metrics_by_tag = evaluator.evaluate()

In [40]:
np.mean([ner_metrics_by_tag[i]['f1'] for i in ner_metrics_by_tag])


KeyError: 'f1'

In [42]:
list(ner_metrics_by_tag.keys())

['PER', 'MISC', 'LOC', 'ORG']

In [45]:
ner_metrics_by_tag

[autoreload of sbe_vallib.validation.scorer.ner_scorer failed: Traceback (most recent call last):
  File "/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/Users/azatsultanov/miniconda/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/azatsultanov/Programming/vallib/repo/

{'PER': {'ent_type': {'correct': 1117,
   'incorrect': 33,
   'partial': 0,
   'missed': 467,
   'spurious': 15,
   'possible': 1617,
   'actual': 1165,
   'precision': 0.9587982832618026,
   'recall': 0.6907854050711194,
   'f1': 0.8030194104960461},
  'partial': {'correct': 1097,
   'incorrect': 0,
   'partial': 53,
   'missed': 467,
   'spurious': 15,
   'possible': 1617,
   'actual': 1165,
   'precision': 0.9643776824034335,
   'recall': 0.6948051948051948,
   'f1': 0.8076923076923076},
  'strict': {'correct': 1069,
   'incorrect': 81,
   'partial': 0,
   'missed': 467,
   'spurious': 15,
   'possible': 1617,
   'actual': 1165,
   'precision': 0.9175965665236051,
   'recall': 0.6611008039579468,
   'f1': 0.7685118619698059},
  'exact': {'correct': 1097,
   'incorrect': 53,
   'partial': 0,
   'missed': 467,
   'spurious': 15,
   'possible': 1617,
   'actual': 1165,
   'precision': 0.9416309012875537,
   'recall': 0.6784168212739641,
   'f1': 0.7886412652767792}},
 'MISC': {'ent_typ